In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-373eac13-4641-43e1-bb7f-6dfaaf91c5d7;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (44ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (30ms)
:: resolution report :: resolve 628ms :: artifacts dl 77ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/24 02:41:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 02:41:39 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
%pip install tensorflow --quiet
%pip install tensorflow_gnn --quiet
%pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.0 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

2023-03-24 02:42:19.197182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 02:42:19.873681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_vi

In [4]:
def getFirstEventFiles():
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(
        Bucket = 'sapient-bucket-trusted',
        Prefix = f'prod/graph/first_events')
    all_files = []
    for content in response.get('Contents', []):
        all_files.append(content['Key'])
        # print(content['Key'])
    files = [f"s3://sapient-bucket-trusted/" + f for f in all_files if 'parquet' in f]

    return files

In [5]:
# https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, test_split=0.1, val_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
# c_col = len(df.columns)

In [7]:
# import pyarrow.parquet as pq
# import s3fs
# s3 = s3fs.S3FileSystem()

In [8]:
# pandas_dataframe = pq.ParquetDataset('s3://sapient-bucket-trusted/prod/graph/first_events', filesystem=s3).read_pandas().to_pandas()

In [9]:
# TENSORFLOW CONFIGURATION
TRAIN_SIZE = 640
SHUFFLE_BUFFER = 500
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
tf.config.threading.set_inter_op_parallelism_threads(16)
pd.set_option('display.max_columns', None)

In [10]:
nonnull_ecar_cols = [
    'id', 'objectID','actorID','object','action','hostname', 'image_path', 
    'parent_image_path', 'new_path', 'file_path', 'malicious'
            ]

In [11]:
lb = LabelBinarizer()
mlb = MultiLabelBinarizer(sparse_output=True)

In [12]:
readFirstEvents().withColumn("malicious", when(col('malicious') == 1, "malicious")
                                            .otherwise("benign")) \
                .groupBy("malicious").count().show()

 2:42AM UTC on Mar 24, 2023 --- read time: 4.292840242385864 seconds ---


+---------+-------+
|malicious|  count|
+---------+-------+
|   benign|4587096|
|malicious|   8596|
+---------+-------+



In [13]:
df = readFirstEvents().select(*nonnull_ecar_cols).cache() \
                    .withColumn("malicious", when(col('malicious') == 1, "malicious")
                                            .otherwise("benign")) \
                    .withColumn('order', row_number().over(Window.partitionBy(lit('1')).orderBy(lit('1')))) \
                    .withColumn('actorUID', dense_rank().over(Window.partitionBy().orderBy('actorID'))-1).orderBy('order') \
                    .withColumn('objectUID', dense_rank().over(Window.partitionBy().orderBy('objectID'))-1).orderBy('order') \
                    .withColumn('UID', dense_rank().over(Window.partitionBy().orderBy('id'))-1).orderBy('order') \
                    .drop('order') \
                    .toPandas()

 2:42AM UTC on Mar 24, 2023 --- read time: 0.241257905960083 seconds ---


In [14]:
# df.isna().sum()

In [15]:
# df = df.fillna('unknown')

In [16]:
df = df.fillna(0)

In [17]:
df.columns

Index(['id', 'objectID', 'actorID', 'object', 'action', 'hostname',
       'image_path', 'parent_image_path', 'new_path', 'file_path', 'malicious',
       'actorUID', 'objectUID', 'UID'],
      dtype='object')

In [18]:
# check number representation worked
df[['actorUID','objectUID', 'UID']].head()

,actorUID,objectUID,UID
0,0,1494892,1641234
1,1,314298,1842141
2,1,473607,3502615
3,1,564548,1574180
4,1,1610008,1734479


In [19]:
num_nodes = len(set(df['actorUID']).union(set(df['objectUID'])))

In [20]:
num_edges = len(set(df['UID']))

In [21]:
num_node_features = len(set(df['parent_image_path']))

In [22]:
# num_edge_features = len( set(df['base_address']).union(set(df['file_path'])).union(set(df['image_path'])) )
num_edge_features = len(set(df['UID']))

In [23]:
node_feature_shape = (num_nodes, num_node_features)
edge_feature_shape = (num_edges, num_edge_features)

In [24]:
# Create a dictionary to store the node and edge features
node_features = {}
for row in df.itertuples():
    source_node = row.actorID
    target_node = row.objectID
    source_node_features = row.UID
    target_node_features = row.objectID
    
    # Add node features for source node
    if source_node not in node_features:
        node_features[source_node] = source_node_features
    
    # Add node features for target node
    if target_node not in node_features:
        node_features[target_node] = target_node_features

In [25]:
edges = []
for row in df.itertuples():
    source_node = row.actorID
    target_node = row.objectID
    edge_features = row.UID
    
    edges.append((source_node, target_node, edge_features))

In [26]:
graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets = {
        "actorUID": tfgnn.NodeSet.from_fields(
            sizes = [len(df)],
            features ={
                'parent_image_path': np.array(df['parent_image_path'],
                                dtype='str').reshape(len(df),1)
            }),
        "objectID": tfgnn.NodeSet.from_fields(
            sizes = [len(df)],
            features ={
                'file_path': np.array(df['file_path'],
                                   dtype='str').reshape(len(df),1),
                'image_path': np.array(df['image_path'],
                                   dtype='str').reshape(len(df),1),
            })
    },
    edge_sets ={
        "id": tfgnn.EdgeSet.from_fields(
            sizes = [len(df)],
            features = {
                'id': np.array(df['UID'],
                                  dtype='str').reshape(len(df),1)},
            adjacency = tfgnn.Adjacency.from_indices(
                source = ("actorUID", np.array(df['actorUID'], dtype='int32')),
                target = ("objectUID", np.array(df['objectUID'], dtype='int32')))),
})

2023-03-24 02:44:46.366025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-24 02:44:46.371205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-24 02:44:46.371842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [27]:
graph_tensor

GraphTensor(
  context=Context(features={}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['actorUID', 'objectID'],
  edge_set_names=['id'])

In [28]:
df = readFirstEvents().withColumn("malicious", when(col('malicious') == 1, "malicious")
                                            .otherwise("benign")) \
                    .withColumn('order', row_number().over(Window.partitionBy(lit('1')).orderBy(lit('1')))) \
                    .withColumn('actorUID', dense_rank().over(Window.partitionBy().orderBy('actorID'))-1).orderBy('order') \
                    .withColumn('objectUID', dense_rank().over(Window.partitionBy().orderBy('objectID'))-1).orderBy('order') \
                    .drop('order') \
                    .toPandas()

 2:44AM UTC on Mar 24, 2023 --- read time: 0.283799409866333 seconds ---


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [29]:
df.count()

id                   4595692
timestamp            4595692
objectID             4595692
actorID              4595692
object               4595692
action               4595692
hostname             4595692
user_name                  0
privileges                 0
image_path           4255823
parent_image_path    1440450
new_path               22696
file_path            2806788
direction                  0
logon_id                   0
requesting_domain          0
requesting_user            0
event_minute         4595692
event_day            4595692
event_hour           4595692
malicious            4595692
relationship         4595692
actorUID             4595692
objectUID            4595692
dtype: int64

In [30]:
df.isna().sum()

id                         0
timestamp                  0
objectID                   0
actorID                    0
object                     0
action                     0
hostname                   0
user_name            4595692
privileges           4595692
image_path            339869
parent_image_path    3155242
new_path             4572996
file_path            1788904
direction            4595692
logon_id             4595692
requesting_domain    4595692
requesting_user      4595692
event_minute               0
event_day                  0
event_hour                 0
malicious                  0
relationship               0
actorUID                   0
objectUID                  0
dtype: int64

In [31]:
df = df.fillna(0)

In [32]:
# Extract the source and destination indices
src_idx = df['actorUID'].tolist()
dst_idx = df['objectUID'].tolist()

In [33]:
# Extract the node features
node_features = df[['parent_image_path', 'file_path']].values

In [34]:
# Extract the edge features
edge_features = df[['id']].values

In [35]:
# Create the sparse adjacency matrix
num_nodes = len(set(src_idx + dst_idx))
indices = list(zip(src_idx, dst_idx))
values = [1] * len(indices)
adj_matrix = tf.sparse.SparseTensor(
    indices=indices,
    values=values,
    dense_shape=[num_nodes, num_nodes]
)

In [36]:
# Create the sparse tensor for the edge features
edge_feature_tensor = tf.SparseTensor(
    indices=indices,
    values=tf.reshape(edge_features, [-1]),
    dense_shape=[num_nodes, edge_features.shape[1]]
)

In [38]:
# sparse_graph = adj_matrix + edge_feature_tensor

In [39]:
nodes = dict()
nodes_list = list(set(df['actorUID'].unique()) | set(df['objectUID'].unique()))
for i, node in enumerate(nodes_list):
    nodes[node] = i

In [40]:
edges = []
for index, row in df.iterrows():
    source = nodes[row['actorUID']]
    destination = nodes[row['objectUID']]
    features = row.id
    edges.append((source, destination, features))

In [41]:
indices = [(edge[0], edge[1]) for edge in edges]
values = [edge[2] for edge in edges]
dense_shape = (len(nodes), len(nodes))

In [42]:
sparse_tensor = tf.SparseTensor(
    indices=indices,
    values=values,
    dense_shape=dense_shape
)

In [43]:
# Create input data
inputs = [[edge[0], edge[1]] for edge in edges]

In [44]:
inputs = tf.ragged.constant(inputs)

In [45]:
event_count = inputs.shape[0]

In [46]:
# Create output data
# outputs = np.array([label])

In [47]:
# Reshape input and output data for LSTM
inputs = tf.expand_dims(inputs, axis=0)
outputs = tf.expand_dims(outputs, axis=0)

NameError: name 'outputs' is not defined

In [48]:
# Define LSTM model
model = keras.Sequential([
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])

In [49]:
# Compile and fit model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(inputs, outputs, epochs=10, batch_size=1)

NameError: name 'outputs' is not defined

Data -> Sparse or Ragged Tensor -> Split -> Input -> Model

In [50]:
x = tf.keras.Input(shape=(4,), sparse=True)
y = tf.keras.layers.Dense(4)(x)

In [51]:
model = tf.keras.Model(x, y)

In [52]:
sparse_data = tf.sparse.SparseTensor(
    indices = [(0,0),(0,1),(0,2),
               (4,3),(5,0),(5,1)],
    values = [1,1,1,1,1,1],
    dense_shape = (6,4)
)

In [53]:
model(sparse_data)

<tf.Tensor: shape=(6, 4), dtype=float32, numpy=
array([[ 0.01671338, -0.19815761, -0.33072704, -0.7705123 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.15675229,  0.803876  ,  0.14709634,  0.16671842],
       [-0.11896354, -0.5517801 ,  0.14482683, -0.6927439 ]],
      dtype=float32)>

In [54]:
model.predict(sparse_data)

1/1 [==============================] - 0s 89ms/step


2023-03-24 02:57:03.472557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [6,3]
	 [[{{node Placeholder/_0}}]]


array([[ 0.01671338, -0.19815761, -0.33072704, -0.7705123 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.15675229,  0.803876  ,  0.14709634,  0.16671842],
       [-0.11896354, -0.5517801 ,  0.14482683, -0.6927439 ]],
      dtype=float32)

In [55]:
data = tf.ragged.constant([ [[940, 203, 668, 387, 790, 320, 939, 185],[315, 515, 791, 181, 939, 787]], 
                             [[564, 205], [820, 180, 993, 739]] ]) 

In [56]:
# https://stackoverflow.com/questions/70399367/sparse-feature-vector-sequence-as-input-to-lstm
X = keras.Input(shape=[inputs.shape[0], inputs.shape[1]], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(inputs.shape[0], 16)(X)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

In [57]:
model = tf.keras.Model(X, output)
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 4595692)]      0         
                                                                 
 embedding (Embedding)       (None, 1, 4595692, 16)    16        
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 1, 16)            0         
 bda)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 32)                6144      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 activation (Activation)     (None, 32)                0         
                                                           

In [58]:
print(model(data))  

2023-03-24 02:57:09.254299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


tf.Tensor(
[[-0.17234889]
 [-0.17234889]], shape=(2, 1), dtype=float32)


In [59]:
model.predict(data)

1/1 [==============================] - 0s 192ms/step


2023-03-24 02:57:09.655249: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [2]
	 [[{{node Placeholder/_0}}]]


array([[-0.17234889],
       [-0.17234889]], dtype=float32)

In [60]:
inputs.shape

TensorShape([1, 4595692, None])

In [65]:
df.count()

2598111

In [67]:
df.columns

['Trace', 'malicious', 'event_sequence']

In [68]:
df.limit(3).toPandas()

,Trace,malicious,event_sequence
0,111669149697,0,"[1000111000000111001000001, 111001001111100110..."
1,111669149698,0,"[1111011111100100111000000, 001001100010110101..."
2,111669149699,0,"[0011010000010001001100010, 011001110100101000..."
